In [2]:
import yfinance as yf
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from io import StringIO
from time import sleep

In [3]:
def get_stock_data_all(stock_ticker):
    ticker = yf.Ticker(stock_ticker)
    price = ticker.history(period = 'max')[['Close', 'Volume', 'Dividends','Stock Splits']]
    price['Stock'] = stock_ticker
    dic = ticker.info
    stock_details = {i : dic.get(i) for i in ['industryKey', 'sectorKey']}
    stock_details['Stock'] = stock_ticker
    stock_news = pd.DataFrame(ticker.news)[['title', 'publisher', 'link', 'type', 'providerPublishTime']]
    stock_news['Stock'] = stock_ticker
    return price, stock_details, stock_news

In [4]:
def get_macro_trends(ticker, stock_name, links):
    dfs = pd.DataFrame()
    driver = webdriver.Chrome()
    marcotrends_url_prefix = "https://www.macrotrends.net/stocks/charts"
    for link in links:
        driver.get(link.format(**{'stock_ticker' : ticker, "stock_name" : stock_name,
                                 "marcotrends_url_prefix" : marcotrends_url_prefix}))
        sleep(2)
        for table in driver.find_elements(By.TAG_NAME, "table"):
            if 'Historical' in table.text or 'Quarterly' in table.text:
                df = pd.read_html(StringIO(table.get_attribute('outerHTML')))[0]
                try:
                    df.columns = df.columns.droplevel(level = 0)
                    df = df[['Date', links.get(link)]]
                except:
                    df.columns = ['Date', links.get(link)]
                df = df.set_index('Date')
                if dfs.empty:
                    dfs = df
                else:
                    dfs = dfs.merge(df, left_index = True, right_index = True, how = 'outer')
                dfs['Stock'] = ticker
                break
    driver.close()
    return dfs

In [5]:
# stock_ticker = 'AAPL'
stock_ticker_map = {'AAPL' : 'apple', 'NVDA' : 'nvidia', 'AMD' : 'amd', 
                    "WFC" : "wells-fargo", "JPM" : "jpmorgan-chase"
}

links = {"{marcotrends_url_prefix}/{stock_ticker}/{stock_name}/pe-ratio" : 'PE Ratio',
        "{marcotrends_url_prefix}/{stock_ticker}/{stock_name}/eps-earnings-per-share-diluted" : "Quarterly EPS",
        "{marcotrends_url_prefix}/{stock_ticker}/{stock_name}/price-book" : 'Price to Book Ratio',
        "{marcotrends_url_prefix}/{stock_ticker}/{stock_name}/roe" : "Return on Equity",
        "{marcotrends_url_prefix}/{stock_ticker}/{stock_name}/price-sales" : "Price to Sales Ratio",
        "{marcotrends_url_prefix}/{stock_ticker}/{stock_name}/shares-outstanding" : "Quarterly Shares Outstanding",
        "{marcotrends_url_prefix}/{stock_ticker}/{stock_name}/revenue" : "Quarterly Revenue"
}

In [6]:
fundamentals_df = pd.DataFrame()
price_df = pd.DataFrame()
details_df = pd.DataFrame()
news_df = pd.DataFrame()

for stock_ticker, stock_name in stock_ticker_map.items():
    print(stock_ticker)
    dfs = get_macro_trends(stock_ticker.replace('.','-'), stock_name, links)
    fundamentals_df = pd.concat([fundamentals_df, dfs])
    lst = get_stock_data_all(stock_ticker)
    price_df = pd.concat([price_df, lst[0]])
    details_df = pd.concat([details_df, pd.DataFrame([lst[1]])])
    news_df = pd.concat([news_df, lst[2]])

AAPL


C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html'

NVDA


C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html'

AMD


C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html'

WFC


C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html'

JPM


C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute('outerHTML'))[0]
C:\Users\ajayd\AppData\Local\Temp\ipykernel_26920\3637846283.py:11: FutureWarning: Passing literal html to 'read_html'

In [8]:
fundamentals_df.to_csv('Financials.csv')
details_df.to_csv('Info.csv', index = False)
news_df.to_csv('News.csv', index = False)
price_df.to_csv('Stock_Price.csv')